In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statistics 
import math

In [ ]:
matches = pd.read_csv("/content/sample_data/matches.csv")
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [ ]:
# setting up the table with relevant columns
dropList = ["result","player_of_match","venue","umpire1","umpire2","umpire3"]
matches.drop(labels=dropList, axis=1, inplace=True)
matches.head() 

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,Sunrisers Hyderabad,35,0
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,0,Rising Pune Supergiant,0,7
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,0,Kolkata Knight Riders,0,10
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,0,Kings XI Punjab,0,6
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,0,Royal Challengers Bangalore,15,0


In [ ]:
matches[pd.isnull(matches['winner'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
300,301,2011,Delhi,2011-05-21,Delhi Daredevils,Pune Warriors,Delhi Daredevils,bat,0,NaN,0,0
545,546,2015,Bangalore,2015-04-29,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,0,NaN,0,0
570,571,2015,Bangalore,2015-05-17,Delhi Daredevils,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,NaN,0,0
744,11340,2019,Bengaluru,30/04/19,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,0,NaN,0,0


In [ ]:
matches['winner'].fillna('Draw', inplace=True)
matches[pd.isnull(matches['winner'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets


In [ ]:
matches[pd.isnull(matches['city'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
461,462,2014,NaN,2014-04-19,Mumbai Indians,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,Royal Challengers Bangalore,0,7
462,463,2014,NaN,2014-04-19,Kolkata Knight Riders,Delhi Daredevils,Kolkata Knight Riders,bat,0,Delhi Daredevils,0,4
466,467,2014,NaN,2014-04-23,Chennai Super Kings,Rajasthan Royals,Rajasthan Royals,field,0,Chennai Super Kings,7,0
468,469,2014,NaN,2014-04-25,Sunrisers Hyderabad,Delhi Daredevils,Sunrisers Hyderabad,bat,0,Sunrisers Hyderabad,4,0
469,470,2014,NaN,2014-04-25,Mumbai Indians,Chennai Super Kings,Mumbai Indians,bat,0,Chennai Super Kings,0,7
474,475,2014,NaN,2014-04-28,Royal Challengers Bangalore,Kings XI Punjab,Kings XI Punjab,field,0,Kings XI Punjab,0,5
476,477,2014,NaN,2014-04-30,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,0,Sunrisers Hyderabad,15,0


In [ ]:
matches['city'].fillna('Dubai', inplace=True)
matches[pd.isnull(matches['city'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets


In [ ]:
matches.replace(["Deccan Chargers","Delhi Daredevils"],["Sunrisers Hyderabad","Delhi Capitals"],inplace=True,regex=True)
match1 = matches[((matches.team1=="Kings XI Punjab") & (matches.team2=="Kolkata Knight Riders")) | ((matches.team1=="Kolkata Knight Riders") & (matches.team2=="Kings XI Punjab"))]
match1.shape[0]

25

In [ ]:
mw_kxip = 0 
mw_kkr = 0
lst= [i for i in match1['winner']] 
print("Win Tracker!")
for i in lst:
  if i=="Kolkata Knight Riders":
    mw_kkr += 1
  else:
    mw_kxip += 1
  print(str(mw_kxip)+" "+str(mw_kkr))
print("KXIP vs KKR : "+str(mw_kxip)+" "+str(mw_kkr))

Win Tracker!
0 1
1 1
2 1
2 2
2 3
3 3
3 4
4 4
4 5
5 5
5 6
6 6
6 7
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
8 14
8 15
8 16
8 17
KXIP vs KKR : 8 17


In [ ]:
last_3_season = matches[(matches['season']>=2017) & np.logical_or(np.logical_and(matches.team1=="Kolkata Knight Riders",matches.team2=="Kings XI Punjab"),np.logical_and(matches.team1=="Kings XI Punjab",matches.team2=="Kolkata Knight Riders"))]
last_3_season.groupby('winner').season.count()

winner
Kings XI Punjab          2
Kolkata Knight Riders    4
Name: season, dtype: int64

Out of 20 matches held between KKR and KXIP, KKR has steady lead against KXIP winning 66% of times in both the scenarios

In [ ]:
deliveries = pd.read_csv("/content/sample_data/deliveries.csv")
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,NaN,NaN,NaN


In [ ]:
dropToBeList = ['inning','is_super_over','bye_runs','legbye_runs','fielder']
deliveries.drop(dropToBeList, axis=1, inplace=True)
deliveries['dismissal_kind'].fillna('Not Out',inplace=True)
deliveries.head()

,match_id,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,wide_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind
0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
2,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,4,0,4,NaN,Not Out
3,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
4,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,2,0,0,0,2,2,NaN,Not Out


In [ ]:
ballbyball = deliveries[((deliveries.batting_team=="Kolkata Knight Riders") & (deliveries.bowling_team=="Kings XI Punjab")) | ((deliveries.batting_team=="Kings XI Punjab") & (deliveries.bowling_team=="Kolkata Knight Riders"))]
ballbyball.head()

,match_id,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,wide_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind
2355,11,Kings XI Punjab,Kolkata Knight Riders,1,1,HM Amla,M Vohra,TA Boult,0,0,0,0,0,0,NaN,Not Out
2356,11,Kings XI Punjab,Kolkata Knight Riders,1,2,HM Amla,M Vohra,TA Boult,0,0,0,4,0,4,NaN,Not Out
2357,11,Kings XI Punjab,Kolkata Knight Riders,1,3,HM Amla,M Vohra,TA Boult,0,0,0,0,0,0,NaN,Not Out
2358,11,Kings XI Punjab,Kolkata Knight Riders,1,4,HM Amla,M Vohra,TA Boult,0,0,0,2,0,2,NaN,Not Out
2359,11,Kings XI Punjab,Kolkata Knight Riders,1,5,HM Amla,M Vohra,TA Boult,0,0,0,4,0,4,NaN,Not Out


In [ ]:
#Q2
match_ids = list(set([i for i in ballbyball['match_id']]))
dots=[]
for i in match_ids:
  dot_balls_in_match = ballbyball[(ballbyball['match_id']==i) & (ballbyball.total_runs==0)].groupby('match_id').total_runs.count()
  dots.append(dot_balls_in_match.values[0])
print(statistics.median(dots))
print(statistics.mean(dots))
print(statistics.median_grouped(dots))

80
80
80.0


There are around 80 dot bolls tomorrow in between KXIP and KKR

In [ ]:
#Q3
dismmissal_count_array=[]
for i in match_ids:
  df = ballbyball[(ballbyball.match_id==i)&(ballbyball.over>=15)&(ballbyball.over<=20)].groupby('match_id').player_dismissed.count()
  dismmissal_count_array.append(df.values[0])
print(statistics.median(dismmissal_count_array))
print(statistics.mean(dismmissal_count_array))
print(statistics.median_grouped(dismmissal_count_array))

5
5
5.0


There might be 4-6 wickets falling in between 15-20 overs summing both the innings

In [ ]:
#Q4
glenn_rpm = ballbyball[ballbyball['batsman']=="GJ Maxwell"].groupby('match_id').batsman_runs.sum().reset_index()
glenn_with_kxip = list(set([i for i in glenn_rpm['match_id']]))
glenn_scores=[i for i in glenn_rpm['batsman_runs']]
team_scores = []
for i in glenn_with_kxip:
  df = ballbyball[(ballbyball.match_id==i) & (ballbyball.batting_team=="Kings XI Punjab")].total_runs.sum()
  team_scores.append(df)
print((statistics.median(glenn_scores)/statistics.median(team_scores))*100)


12.82051282051282


Glenn generally contributes to 10-15% of his team score!

In [ ]:
#Q5
nobs_in_match = ballbyball[ballbyball.noball_runs!=0].groupby('match_id').noball_runs.count()
print(statistics.median(nobs_in_match))
print(statistics.median_grouped(nobs_in_match))

1
1.1428571428571428
